In [1]:
import tweepy
import constants
import pandas as pd
import datetime as dt

### Import Keys & Create Tweepy Client

In [2]:

bearer_token = constants.TWITTER_BEARER_TOKEN

client = tweepy.Client(bearer_token=bearer_token)



In [3]:
from team_queries import team_queries
team_queries[str('bills')]

'(#billsmafia OR #gobills OR #bills) -is:retweet lang:en -#dirtybirds -#falcons -#birdcityfootball -#cardinals -#ravensflock -#ravens -#keeppounding -#panthers -#dabears -#bearsnation -#bears -#rulethejungle -#bengals -#whodey -#browns -#clevelandbrowns  -#dawgpound -#dallascowboys -#cowboys -#broncoscountry -#letsride -#broncos -#onepride -#lions  -#gopackgo -#packers -#wearetexans -#texans -#fortheshoe -#colts -#duuuval -#jags -#jaguars -#chiefskingdom -#chiefs'

#### Individual Team Search - No Pagination
Can use for testing

In [4]:
# build search tweets 
# will use paginate normally, test case
user_fields = ["id"]
expansions = ["author_id"]

# max_results min = 10
def build_team_search(team_query, max_results = 10):
    team_response = client.search_recent_tweets(
            team_query, 
            max_results=max_results, 
            tweet_fields=["author_id", "created_at", "text"], 
            user_fields=user_fields, 
            expansions=expansions
        )
    return team_response

bills_response = build_team_search(team_queries[str('bills')], max_results=10)
bills_response.data

[<Tweet id=1599840401468235776 text='We have 12 days until kickoff. This forecast will change 200 times until then. And I expect to see someone post every single one 😂😂 it is great #BillsMafia https://t.co/9mbI5l6SXA'>,
 <Tweet id=1599840307754827776 text='Just a reminder for #billsmafia not winning MVP doesnt make Josh Allen any less elite.'>,
 <Tweet id=1599840253128617984 text='I unironically want there to be a severe blizzard just before kickoff of this game for... reasons... #BillsMafia https://t.co/fCNB0qqJnr'>,
 <Tweet id=1599840162988507144 text='@MadGlab Kyle Williams, forever and always. #GoBills https://t.co/VuioCnw0pO'>,
 <Tweet id=1599839901146415106 text='@IsaiahHodgins Congrats!  Good luck is stretch run... from #billsmafia'>,
 <Tweet id=1599839589597732864 text='Here we go……@MiamiDolphins you might want to bundle up….it might be chilly 🥶 ❄️. Actually we’re hoping for a blizzard…..you cooked us in the heat so only right we freeze your 🥜 off. Bills by a Billion #Billsmafi

### Get times and games from schedule data

1. Get games from schedule
2. For each game
    a. get both teams
    b. get date & time
    c. get week
    d. get home and away ? not sure need this
3. For each game create database of tweets for each team
    a. build query
        i. Queries are always the same, can just create a dict of queries
    b. get start time (4 days before game?) and end time (1 hr before game)
    b. pagination function
        ii. start time & end time
    b. create database of tweets

### Import Schedule CSV

In [6]:

def import_schedule_csv(filepath):
    sched_df = pd.read_csv(filepath, infer_datetime_format=True)

    # convert all datetime fields to datetime type
    sched_df['GameTime'] = pd.to_datetime(sched_df['GameTime'])
    sched_df['SearchStartTime'] = pd.to_datetime(sched_df['SearchStartTime'])
    sched_df['SearchEndTime'] = pd.to_datetime(sched_df['SearchEndTime'])

    # convert datetime to twitter appropriate format for datetime requests
    sched_df['GameTime'] = sched_df['GameTime'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')
    sched_df['SearchStartTime'] = sched_df['SearchStartTime'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')
    sched_df['SearchEndTime'] = sched_df['SearchEndTime'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')

    return sched_df


In [7]:
filepath_13 = './schedules/schedule_wk_13_cleaned.csv'

sched_13_df = import_schedule_csv(filepath_13)
sched_13_df

,Week,Day,Date,Time,home,away,home_score,away_score,GameTime,SearchStartTime,SearchEndTime
0,13,Thu,2022-12-01,8:15 PM,patriots,bills,10,24,2022-12-02T01:15:00Z,2022-11-28T00:15:00Z,2022-12-02T00:15:00Z
1,13,Sun,2022-12-04,1:00 PM,texans,browns,14,27,2022-12-04T18:00:00Z,2022-11-30T17:00:00Z,2022-12-04T17:00:00Z
2,13,Sun,2022-12-04,1:00 PM,ravens,broncos,10,9,2022-12-04T18:00:00Z,2022-11-30T17:00:00Z,2022-12-04T17:00:00Z
3,13,Sun,2022-12-04,1:00 PM,lions,jaguars,40,14,2022-12-04T18:00:00Z,2022-11-30T17:00:00Z,2022-12-04T17:00:00Z
4,13,Sun,2022-12-04,1:00 PM,vikings,jets,27,22,2022-12-04T18:00:00Z,2022-11-30T17:00:00Z,2022-12-04T17:00:00Z
5,13,Sun,2022-12-04,1:00 PM,giants,commanders,20,20,2022-12-04T18:00:00Z,2022-11-30T17:00:00Z,2022-12-04T17:00:00Z
6,13,Sun,2022-12-04,1:00 PM,eagles,titans,35,10,2022-12-04T18:00:00Z,2022-11-30T17:00:00Z,2022-12-04T17:00:00Z
7,13,Sun,2022-12-04,1:00 PM,falcons,steelers,16,19,2022-12-04T18:00:00Z,2022-11-30T17:00:00Z,2022-12-04T17:00:00Z
8,13,Sun,2022-12-04,1:00 PM,bears,packers,19,28,2022-12-04T18:00:00Z,2022-11-30T17:00:00Z,2022-12-04T17:00:00Z
9,13,Sun,2022-12-04,4:05 PM,49ers,dolphins,33,17,2022-12-04T21:05:00Z,2022-11-30T20:05:00Z,2022-12-04T20:05:00Z


# Pagination

Requests limited to 100 tweets at a time

Must paginate requests using ```tweepy.Paginator```   
```limit```: sets how many pages  
```max_results```: sets how many results per page (limited to 100)  
Total Results = limit * max_results  

```start_time```: start of when to retrieve tweets  
```end_time```: end of when to retrieve tweets  
**Note**: times are in format of ```2022-11-24T15:25:00Z```  
This is ZULU time which is **7 hours ahead of MST**  


### Individual Team Paginator Search

In [15]:
#check start time within 7 days

def check_start_time_not_greater_7_days(start_time):
    week_ago = dt.datetime.utcnow() - pd.to_timedelta('7 days')
    # print('1 week ago:', week_ago, '\t Start_time: ', start_time)
    check = week_ago < dt.datetime.strptime(start_time, '%Y-%m-%dT%H:%M:%SZ')
    # if (check == False):
    #     print('Start time too old')
    # else:
    #     print('Start time allowed')
    return check

check_start_time_not_greater_7_days('2022-11-23T17:55:00Z')

False

In [18]:

# start_time=2019-01-01T17:00:00Z
# end_time=2020-12-12T01:00:00Z
#2022-11-23T20:05:00Z	

def paginate_search(team_query, max_results, start_time, end_time, limit=20):
    start_valid = check_start_time_not_greater_7_days(start_time)
    tweet_fields = ["author_id", "created_at", "text"]
    
    if (start_valid == True):
        paginator = tweepy.Paginator(
            client.search_recent_tweets, 
            query = team_query, 
            tweet_fields = tweet_fields, 
            start_time=start_time, 
            end_time=end_time,
            max_results = max_results,
            limit = limit
        )
        return paginator
    else:
        # run query with start time being one week ago + 1 minute
        # otherwise time becomes invalid by time last pages come through, need the buffer
        start_time = dt.datetime.utcnow() - pd.to_timedelta('7 days') + pd.to_timedelta('1 minute')
        print('Start Time Not Valid', team_query[2:30], '\n New Sart Time: ', start_time)
        paginator = tweepy.Paginator(
            client.search_recent_tweets, 
            query = team_query, 
            tweet_fields = tweet_fields, 
            start_time=start_time, 
            end_time=end_time,
            max_results = max_results,
            limit = limit
        )
        return paginator

    

### Call Paginator Search from Game Schedule
Iterate through each game row  
Get all data needed  
Call paginae search for both home and away teams  
Add to dictionary

In [10]:
def get_paginator_dict_key(wk, team):
    team_key = 'wk_' + str(wk) + '_' + team
    return team_key

def create_week_paginations_from_schedule(wk_sched_df, team_queries, max_results = 100, limit = 50):
    # create empty paginators dict
    paginators = {}

    # iterate through each game row
    for game_row in wk_sched_df.itertuples():
        wk = game_row.Week

        # get search start and end times
        
        start_time = game_row.SearchStartTime
        end_time = game_row.SearchEndTime
        
        #get home and away teams
        home_team = game_row.home
        away_team = game_row.away

        # get team queries
        home_team_query = team_queries[str(home_team)]
        away_team_query = team_queries[str(away_team)]

        # create team paginators for home and away
        home_team_paginator = paginate_search(
            home_team_query, 
            max_results, 
            start_time, 
            end_time,
            limit
        )
        away_team_paginator = paginate_search(
            away_team_query, 
            max_results, 
            start_time, 
            end_time,
            limit
        )

        # create dict key from         
        home_key = get_paginator_dict_key(wk, home_team)
        away_key = get_paginator_dict_key(wk, away_team)

        # add paginators to dict
        paginators[home_key] = {
            "paginator": home_team_paginator,
            "team": home_team
        }
        paginators[away_key] = {
            "paginator": away_team_paginator,
            "team": away_team
        }
        
    return paginators

### Create CSV file for team
- Tweet Team
- Tweet Text
- Tweet Author
- Tweet Date Time

In [11]:
import csv
import tweepy

def create_tweet_save_path (week, key):
    return  f'./team_data/wk_{week}/{key}_tweets.csv'


# 'team_data/%s_tweets.csv' % (key)
def create_dataset(paginator_dict, key, week_num):\
    # get team and paginator
    team = paginator_dict[str("team")]
    paginator = paginator_dict[str("paginator")]

    # create save path for tweet csv's
    file_save_path = create_tweet_save_path(week_num, key)
    
    with open(file_save_path, 'w', encoding="utf-8") as file:
        w = csv.writer(file)
        
        # Write header row (feature column names of your choice)
        w.writerow(['team',
                     'timestamp', 
                     'tweet_text', 
                     'userid' 
                     ])
        # initiate tweet and page counts
        tweet_count = 0
        page_count = 0
        
        # iterate through pages of tweets
        for page in paginator: 
            page_count += 1

            # if no data found record and skip
            if (page.data == None):
                print(f'NO DATA FOR {team}')
                print(page.meta)
            
            else:
                # For each tweet add to csv file
                for tweet in page.data:
                    tweet_count += 1
                
                    w.writerow([team,
                                tweet.created_at, 
                                tweet.text.replace('\n',' ').encode('utf-8'), 
                                tweet.author_id, 
                                ])
        # record data
        print(f'Team: {team} \t\t Page Count: {page_count} \t Tweet Count: {tweet_count}')

In [12]:
def create_all_team_datasets(team_paginators, week_num):
    # iterate through team paginator dict
    for key, paginator_dict in team_paginators.items():
        # print('KEY:%s - PAGINATOR: %s ' %(key, paginator_dict))
        # pass each to create_dataset
        create_dataset(paginator_dict, key, week_num)

### Run Code to get all Datasets

In [21]:
sched_13_df_thurs = sched_13_df.iloc[:1]
sched_13_df_no_monday = sched_13_df.iloc[8:]
sched_13_df_no_monday

,Week,Day,Date,Time,home,away,home_score,away_score,GameTime,SearchStartTime,SearchEndTime
8,13,Sun,2022-12-04,1:00 PM,bears,packers,19,28,2022-12-04T18:00:00Z,2022-11-30T17:00:00Z,2022-12-04T17:00:00Z
9,13,Sun,2022-12-04,4:05 PM,49ers,dolphins,33,17,2022-12-04T21:05:00Z,2022-11-30T20:05:00Z,2022-12-04T20:05:00Z
10,13,Sun,2022-12-04,4:05 PM,rams,seahawks,23,27,2022-12-04T21:05:00Z,2022-11-30T20:05:00Z,2022-12-04T20:05:00Z
11,13,Sun,2022-12-04,4:25 PM,bengals,chiefs,27,24,2022-12-04T21:25:00Z,2022-11-30T20:25:00Z,2022-12-04T20:25:00Z
12,13,Sun,2022-12-04,4:25 PM,raiders,chargers,27,20,2022-12-04T21:25:00Z,2022-11-30T20:25:00Z,2022-12-04T20:25:00Z
13,13,Sun,2022-12-04,8:20 PM,cowboys,colts,54,19,2022-12-05T01:20:00Z,2022-12-01T00:20:00Z,2022-12-05T00:20:00Z


In [22]:
week_num = '13'
paginators_wk13 = create_week_paginations_from_schedule(sched_13_df_no_monday, team_queries, 100)
create_all_team_datasets(paginators_wk13, week_num)

Team: bears 		 Page Count: 26 	 Tweet Count: 2597
Team: packers 		 Page Count: 28 	 Tweet Count: 2753
Team: 49ers 		 Page Count: 36 	 Tweet Count: 3500
Team: dolphins 		 Page Count: 50 	 Tweet Count: 4992
Team: rams 		 Page Count: 13 	 Tweet Count: 1232
Team: seahawks 		 Page Count: 10 	 Tweet Count: 984
Team: bengals 		 Page Count: 30 	 Tweet Count: 2986
Team: chiefs 		 Page Count: 29 	 Tweet Count: 2860
Team: raiders 		 Page Count: 31 	 Tweet Count: 3068
Team: chargers 		 Page Count: 15 	 Tweet Count: 1430
Team: cowboys 		 Page Count: 32 	 Tweet Count: 3102
Team: colts 		 Page Count: 11 	 Tweet Count: 1061


## Need to filter out multiteam hashtags